# Wflow SBM Model Build & Simulation

## Physically-Based Distributed Hydrological Modeling for the Upper Niger Basin

**Objective:** Construct a spatially distributed rainfall-runoff model using [Wflow SBM](https://deltares.github.io/Wflow.jl/stable/) to simulate discharge dynamics driven by ERA5-Land forcing data.

---

### Model Architecture: Wflow SBM

The **Soil-Bucket Model (SBM)** is a conceptual vertical water balance model coupled with:
- **Kinematic wave** routing for surface and subsurface lateral flow
- **Snow/glacier** modules (optional, disabled for tropical Niger)

**Key Fluxes:**
$$P \rightarrow \text{Interception} \rightarrow \text{Infiltration} \rightarrow \text{Percolation} \rightarrow Q_{river}$$

Where:
- $P$: Precipitation (ERA5-Land)
- $ET$: Evapotranspiration (driven by PET from ERA5)
- $Q_{river}$: Simulated discharge

### Why Wflow for Flood Forecasting?
1. **Distributed**: Captures spatial heterogeneity in rainfall and terrain
2. **Physically-based**: Parameters linked to measurable soil/land properties
3. **Operationally proven**: Used by Deltares for real-time flood forecasting

---


# 1. Environment Setup

In [ ]:
import os
import sys
import subprocess
from pathlib import Path
import yaml

import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd

import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Verify HydroMT installation
result = subprocess.run(["hydromt", "--models"], capture_output=True, text=True)
print(result.stdout)
if "wflow" not in result.stdout:
    raise RuntimeError("hydromt_wflow not installed. Run: pip install hydromt_wflow")

# 2. Study Area Definition

The Upper Niger Basin spans Guinea, Mali, and Côte d'Ivoire. We define the model domain using:
- **Subbasin outlet**: A point on the Niger River
- **Stream order**: Minimum Strahler order to include
- **Bounding box**: Spatial extent constraint

In [ ]:
# ============================================================
# REGION DEFINITION
# ============================================================
# Outlet coordinates (longitude, latitude) on Niger River
OUTLET = [-8.5, 11.5]

# Strahler stream order threshold (4 = include major tributaries)
STREAM_ORDER = 4

# Bounding box [West, South, East, North] - matches ERA5 acquisition
BOUNDS = [-10.0, 10.0, -7.0, 12.5]

# Model output directory
MODEL_DIR = Path("../models/wflow_sbm/wflow_niger")
MODEL_DIR.mkdir(parents=True, exist_ok=True)

print(f"Outlet: {OUTLET}")
print(f"Bounds: {BOUNDS}")
print(f"Output: {MODEL_DIR.resolve()}")

# 3. Configuration Files

HydroMT uses two YAML configuration files:
1. **`wflow_build.yml`**: Defines the model setup workflow
2. **`hydromt_data.yml`**: Data catalog pointing to local ERA5 forcing

In [ ]:
# Display build configuration
config_path = Path("../models/wflow_sbm/wflow_build.yml")

print("=" * 60)
print("WFLOW BUILD CONFIGURATION")
print("=" * 60)
with open(config_path) as f:
    print(f.read())

In [ ]:
# Display data catalog
catalog_path = Path("../models/wflow_sbm/hydromt_data.yml")

print("=" * 60)
print("HYDROMT DATA CATALOG")
print("=" * 60)
with open(catalog_path) as f:
    print(f.read())

# 4. Model Build Execution

**HydroMT Command Structure:**
```
hydromt build wflow <output_dir> \
    -r "<region_dict>" \
    -i <build_config.yml> \
    -d <data_catalog.yml> \
    -d artifact_data \
    -vv
```

**Flags:**
- `-r`: Region definition (JSON-like dict)
- `-i`: Build configuration (setup methods)
- `-d`: Data catalogs (can stack multiple)
- `-vv`: Verbose output for debugging

In [ ]:
# Construct the HydroMT build command
region = f"{{'subbasin': {OUTLET}, 'strord': {STREAM_ORDER}, 'bounds': {BOUNDS}}}"

cmd = [
    "hydromt", "build", "wflow",
    str(MODEL_DIR),
    "-r", region,
    "-i", str(config_path),
    "-d", str(catalog_path),
    "-d", "artifact_data",
    "-vv"
]

print("Command:")
print(" ".join(cmd))
print()

# Execute (set RUN_BUILD=True to actually run)
RUN_BUILD = False

if RUN_BUILD:
    print("Building model...")
    result = subprocess.run(cmd, capture_output=True, text=True)
    print(result.stdout)
    if result.returncode != 0:
        print("STDERR:", result.stderr)
else:
    print("Set RUN_BUILD=True to execute the model build.")
    print("This requires ~5-10 minutes and downloads ~500MB of remote data.")

# 5. Model Inspection: Static Maps

After building, HydroMT generates:
- **`staticmaps.nc`**: Gridded parameters (DEM, soil, land use, etc.)
- **`staticgeoms/`**: Vector geometries (basin, rivers, gauges)
- **`wflow_sbm.toml`**: Runtime configuration for Julia Wflow

In [ ]:
# Load static maps (if model exists)
static_path = MODEL_DIR / "staticmaps.nc"

if static_path.exists():
    ds_static = xr.open_dataset(static_path)
    print(f"Static maps loaded: {list(ds_static.data_vars)[:10]}...")
    print(f"Grid shape: {ds_static.dims}")
else:
    print(f"Model not built yet. Run HydroMT build first.")
    ds_static = None

In [ ]:
# Visualize key static parameters
if ds_static is not None:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    plt.suptitle("Wflow Static Parameters", fontsize=14)

    # DEM
    ds_static["wflow_dem"].plot(ax=axes[0,0], cmap="terrain", cbar_kwargs={"label": "m"})
    axes[0,0].set_title("Digital Elevation Model")

    # River network
    ds_static["wflow_river"].plot(ax=axes[0,1], cmap="Blues", cbar_kwargs={"label": ""})
    axes[0,1].set_title("River Network")

    # Soil thickness
    if "SoilThickness" in ds_static:
        ds_static["SoilThickness"].plot(ax=axes[1,0], cmap="YlOrBr", cbar_kwargs={"label": "mm"})
        axes[1,0].set_title("Soil Thickness")

    # Land use (Kc - crop coefficient)
    if "kc" in ds_static:
        ds_static["kc"].isel(time=6).plot(ax=axes[1,1], cmap="Greens", cbar_kwargs={"label": ""})
        axes[1,1].set_title("Crop Coefficient (July)")

    plt.tight_layout()
    plt.show()

# 6. Model Inspection: Forcing Data

The forcing file (`inmaps.nc`) contains the meteorological drivers resampled to the model grid.

In [ ]:
# Load forcing (if model exists)
forcing_path = MODEL_DIR / "inmaps.nc"

if forcing_path.exists():
    ds_forcing = xr.open_dataset(forcing_path)
    print(f"Forcing variables: {list(ds_forcing.data_vars)}")
    print(f"Time range: {ds_forcing.time.values[0]} to {ds_forcing.time.values[-1]}")
    print(f"Timesteps: {len(ds_forcing.time)}")
else:
    print("Forcing not found. Build model first.")
    ds_forcing = None

In [ ]:
# Time series validation
if ds_forcing is not None:
    # Spatial mean time series
    ts_p = ds_forcing["precip"].mean(dim=["x", "y"])
    ts_t = ds_forcing["temp"].mean(dim=["x", "y"]) if "temp" in ds_forcing else None
    ts_pet = ds_forcing["pet"].mean(dim=["x", "y"]) if "pet" in ds_forcing else None

    fig = make_subplots(specs=[[{"secondary_y": True}]])

    fig.add_trace(
        go.Bar(x=ts_p.time.values, y=ts_p.values, name="Precip (mm/day)", marker_color="#3b82f6", opacity=0.6)
    )

    if ts_t is not None:
        fig.add_trace(
            go.Scatter(x=ts_t.time.values, y=ts_t.values, name="Temp (°C)", line=dict(color="#ef4444")),
            secondary_y=True
        )

    if ts_pet is not None:
        fig.add_trace(
            go.Scatter(x=ts_pet.time.values, y=ts_pet.values, name="PET (mm/day)", line=dict(color="#f97316", dash="dot"))
        )

    fig.update_layout(title="Forcing Time Series (Spatial Mean)", height=450, template="plotly_white")
    fig.update_yaxes(title_text="Fluxes (mm/day)", secondary_y=False)
    fig.update_yaxes(title_text="Temperature (°C)", secondary_y=True)
    fig.show()

# 7. Run Wflow Simulation

The simulation is executed using Julia:
```bash
julia run_wflow.jl wflow_niger/wflow_sbm.toml
```

**Prerequisites:**
1. Julia installed with `Wflow.jl` package
2. Model built (staticmaps.nc, inmaps.nc exist)

In [ ]:
# Check TOML configuration
toml_path = MODEL_DIR / "wflow_sbm.toml"

if toml_path.exists():
    print("TOML Configuration (first 50 lines):")
    print("=" * 50)
    with open(toml_path) as f:
        for i, line in enumerate(f):
            if i < 50:
                print(line, end="")
            else:
                print("...")
                break
else:
    print("TOML not found. Build model first.")

In [ ]:
# Run simulation (requires Julia + Wflow.jl)
RUN_SIMULATION = False

if RUN_SIMULATION and toml_path.exists():
    print("Running Wflow simulation...")
    result = subprocess.run(
        ["julia", "../models/wflow_sbm/run_wflow.jl", str(toml_path)],
        capture_output=True,
        text=True
    )
    print(result.stdout)
    if result.returncode != 0:
        print("STDERR:", result.stderr)
else:
    print("Set RUN_SIMULATION=True to execute (requires Julia + Wflow.jl)")

# 8. Simulation Results Analysis

Wflow outputs:
- **`output.nc`**: Gridded time series (discharge, storage, etc.)
- **`output.csv`**: Scalar time series at gauge locations

In [ ]:
# Load discharge output
csv_path = MODEL_DIR / "run_default" / "output.csv"

if csv_path.exists():
    df_q = pd.read_csv(csv_path, parse_dates=["time"], index_col="time")
    print(f"Discharge record: {df_q.index[0]} to {df_q.index[-1]}")
    print(f"Columns: {list(df_q.columns)}")
    
    # Plot
    fig = go.Figure()
    for col in df_q.columns:
        if "Q" in col or "q" in col.lower():
            fig.add_trace(go.Scatter(x=df_q.index, y=df_q[col], name=col))
    
    fig.update_layout(
        title="Simulated Discharge",
        xaxis_title="Date",
        yaxis_title="Discharge (m³/s)",
        template="plotly_white",
        height=400
    )
    fig.show()
else:
    print("No simulation output found. Run simulation first.")

# 9. Summary & Next Steps

## Workflow Complete
1. ✅ Configuration files prepared (`wflow_build.yml`, `hydromt_data.yml`)
2. ✅ HydroMT build command ready
3. ✅ Simulation script (`run_wflow.jl`) prepared

## Next Steps for Flood Forecasting
1. **Calibration**: Adjust parameters (KsatHorFrac, cf_soil) using observed discharge
2. **Threshold Definition**: Define flood threshold (e.g., Q > 99th percentile)
3. **ML Integration**: Use simulated Q as training target for forecast model

## Files Generated
```
models/wflow_sbm/
├── wflow_build.yml       # HydroMT configuration
├── hydromt_data.yml      # Data catalog
├── run_wflow.jl          # Julia simulation script
└── wflow_niger/          # Model instance (after build)
    ├── staticmaps.nc
    ├── inmaps.nc
    ├── wflow_sbm.toml
    └── run_default/      # Simulation outputs
```
